<p style="text-align: center">
    <b><span style="float: center; font-size: 12pt">LocalThingsNetwork</span>
    <br>
    <span style="font-size: 40pt"> Server demonstration </span></b>
</p>

---

This is a demonstration of the `localthingsnet.network.Server` class. The server communicates with multiple clients by exchanging data, evaluating data requests and executing commands. The server can interact with clients through one or multiple sockets if needed. Additionally the server provides extendable routine services for repetitive tasks like data synchronization or ping checking. This demonstration explains the **setup** of a server with its **services**, **servercommands** and **requestables**, how server and clients **interact** with each other and finally in which ways the server can be **modified** for certain needs.

---

In [1]:
import localthingsnet.network as ltn

In [2]:
# Initialise two clients for demonstration purposes
# NOTE: The output of the clients are muted to focus on the server
client1 = ltn.Client('client_1')
client2 = ltn.Client('client_2')

def mute(self, *message):
    pass
client1.printInfo = mute.__get__(client1, ltn.Client)
client1.statusInfo = mute.__get__(client1, ltn.Client)
client1.warningInfo = mute.__get__(client1, ltn.Client)
client1.errorInfo = mute.__get__(client1, ltn.Client)
client2.printInfo = mute.__get__(client2, ltn.Client)
client2.statusInfo = mute.__get__(client2, ltn.Client)
client2.warningInfo = mute.__get__(client2, ltn.Client)
client2.errorInfo = mute.__get__(client2, ltn.Client)

---
---

# ***Server concept***

Based on the python <a href="https://docs.python.org/3/library/socket.html">socket</a> library the server binds a socket to a free port on the executing machine. This socket is referred to as ***serversock***. It is used to connect and register new clients and processes data, mainly text sent from the user. The server can also own a ***maindatasock*** and other **datasocks**. The maindatasock is used for background data transmissions *(f.e. data synchronization)*. It is not necessarily needed but reduces the workload on the serversock. Other datasocks can be defined if needed and customized for certain purposes *(f.e. receiving images or send-receive in a certain pattern)*. Every datasock has a certain receiving function *(recvFunc)* that handles receiving and processing data. A new client can only connect to the serversock through which connections to other datasocks get established. During registration of a new client the maindatasock will be connected and the client gets a *clientid* through which the client can be addressed.

---

# ***Server initialization***

The initialization of a server requires no arguments, but there are some optional arguments:

>***servername***  
> * An identifiable name for the server
> * (Default is 'server')

>***description***  
> * Short description about the abilities or purpose
> * (Default is 'None')

>***adminkey***  
> * Password to gain administration permissions
> * (Default is '', every client gets admin permissions)

>***max_datasize***  
> * Maximum bytes that can be exchanged in one transmission
> * (Default is 1024)

>***preferredport***  
> * Preferred port(s) to bind the serversock to
> * (Default is '>4000')

>***preferreddataport***
> * Preferred port(s) to bind the datasock to
> * (Default is '<3999')

>***logfile***  
> * Creates a file that saves occurred events
> * (Default is '', No logfile gets created)

>***ansi***  
> * Allow ANSI text formatting
> * (Default is True)

<br>

>***NOTE:** It is suggested to use port 4000 and above for serversocks and below 4000 for (main-)datasocks.*

In [3]:
server = ltn.Server(
    servername='Demoserver',
    preferredport='>4000',
    preferreddataport='<3999',
    max_datasize=1024,
    adminkey='DemoKey',
    description="Demonstrate abilities of the localthingsnet(work) project",
    logfile="",
    ansi=True)

server.__version__

'7.43.162'

The server can be started with `startServer()`. This will bind the server *(serversock)* to the first available port in the allowed **port** argument. **Port** can be a single port, a selection of ports or an expression for a port and all lower/higher ports *(f.e. '>4000', Port 4000 is inclusive)*. By default **port** will be the *preferredport* set with server initialization. The same applies for **dataport** which contains allowed ports for datasocks. If wanted the *maindatasock* and the *services* routine can be initialized. Both can also be initialized later on. After the startup clients will be able to connect to the server.

In [4]:
server.startServer(port='>4000', dataport='preferred', maindatasock=False, services=False)

Serversock bound to 4000
Server is running at ('192.168.178.140', 4000)


---

# ***Connect clients***

A client that connects to the serversock passes `registrateClient()` which checks compatibility, exchanges metadata, optionally connects the main- and other datasocks and assigns a *clientid* to the connecting client. All connected clients and their data are stored in the `server.conns` dictionary addressable through a clientid. The client also gets a username that is shown whenever a message is sent. The username is also used to address the client in servercommands. The username can be changed, the clientid not. If the clientid is unknown it can be gained with `getIDof()` and a property of the client.

Below two clients connect to the server and get registered as *client_1* and *client_2*.<br>
*(For more details on how to connect clients refer to the Client demonstration notebook)*

In [5]:
# Connect clients to server
info = client1.connect(server.addr)
info = client2.connect(server.addr)

192.168.178.140 is registered as 'client_1'
192.168.178.140 is registered as 'client_2'


In [6]:
# Get client ids
cl1_id = server.getIDof(username='client_1')
cl2_id = server.getIDof(username='client_2')

In [7]:
print(server.conns[cl1_id].keys())
print(server.conns[cl1_id]['name'])

dict_keys(['name', 'socks', 'isserver', 'addr', 'clientdata', 'permissions', 'muted', 'connecttime', 'ping', 'data'])
client_1


During registration of a connection client and server exchange **client-** and **serverdata** with contains attributes that can be of use to the other instance *(like available commands and requestables, description, sockets of the server, etc.)*. If the server *services* routine is enabled these attributes are constantly synchronized, otherwise values may be outdated when accessed and need to updated via requests. 

In [8]:
print("Serverdata stored in client:\n", client1.cl_serverdata.keys())
print("Clientdata stored in server:\n", server.conns[cl1_id]['clientdata'].keys())

Serverdata stored in client:
 dict_keys(['servername', 'description', 'server_version', 'starttime', 'layer', 'separator', 'max_datasize', 'max_username_length', 'servercommands', 'requestables', 'socks'])
Clientdata stored in server:
 dict_keys(['description', 'client_version', 'commands', 'requestables'])


---

# ***Interact with clients***

The server receives data sent by clients through receiving threads. These threads get started for every client-socket that connects to any sock of the server. The receiving threads loop a *receiving function* ***(recvFunc)*** which receives and processes a single data packet. From there on receiving is done automatically. The *recvFunc* can be set for any datasocket individually when the socket gets initiated. *(More in the datasocks section.)* The default *recvFunc* is `recvClientData()` which processes text, servercommands and requestables. Sending to a client can be done with `sendDataTo()`, `sendMsgTo()`, `sendCommandTo()` and `sendRequestTo()`. These require a clientid or a socket object of the client. Clientids can be gained with `getIDof()` while a client sockets can be gained from `server.conns[*clientid*]['socks'][*port*]`, as shown below. `SendDataTo()` allows to send data to a client, the other functions are just extensions of that. 

> ***Server.sendDataTo(clientsocket: socket, data: object)***
> - Sends a <a href=https://docs.python.org/3/library/pickle.html>picklable</a> python object to a certain client socket.

> ***Server.sendMsgTo(clientid: str, message: str)***
> - Sends a string message through the serversock that gets displayed in the client console.

> ***Server.sendCommandTo(clientsocket: socket, command: str, arguments: list = None)***
> - Calls a predefined clientcommand that gets executed on the client machine
> - Available clientcommands can be viewed in `server.conns[*id*]['clientdata']`

>***Server.sendRequestTo(clientsocket: socket, requested: str, timeout: float = 1.0, get_time: bool = False ) -> object***
> - Requests data from the client
> - Available requestables can be viewed in `server.conns[*id*]['clientdata']`

In [9]:
# Receive data from a client
maindataport = server.getMainDataPort()
client1.sendMsg("Hello server!")
client1.sendData(client1.clientsocks[maindataport], "Send though maindatasock")
client1.sendData(client1.clientsocks[maindataport], [1, 2, 3])

[13:52:13]client_1: Hello server!
[13:52:13]client_1: Send though maindatasock
WAR: Could not identify the purpose of a <class 'list'> obj sent by user 'client_1'


> ***Note:** The code above outputs a warning since the default recvFunc is not suited for data other than strings. Likewise the client doesn't output received datatypes other than strings at all. This can be changed with a custom receiving function (recvFunc)*

In [10]:
# Send data to a client

# Unmute clients to see what it is receiving
client1.printInfo = ltn.Client.printInfo.__get__(client1, ltn.Client)

# Send messages and data to the client
server.sendMsgTo(cl1_id, "Hello client!")
server.sendDataTo(server.conns[cl2_id]['socks'][server.addr[1]], [0, 1, 2, False])

# Mute client again
client1.printInfo =  mute.__get__(client1, ltn.Client)

Hello client!


In [11]:
# Get available clientcommands and -requestables
print(server.conns[cl1_id]['clientdata']['commands'])
print(server.conns[cl1_id]['clientdata']['requestables'])

['changename', 'setlayer', 'updateserverdata', 'disconnect', 'connect', 'newdatasock']
['PING', 'CLIENTDATA']


In [12]:
# Send a (client)command to a client
print("Username before:", client2.username)
server.sendCommandTo(server.conns[cl2_id]['socks'][server.addr[1]], 'changename', 'better name')

Username before: client_2


In [13]:
print("Username after:", client2.username)

Username after: better name


In [14]:
# Request data from a client
clientdata = server.sendRequestTo(server.conns[cl1_id]['socks'][server.addr[1]], 'CLIENTDATA', timeout=1)
print(clientdata)

ping = server.sendRequestTo(server.conns[cl1_id]['socks'][server.addr[1]], 'PING', get_time=True)[1]
print(f"\nPing is: {int(ping * 1000)}ms")

{'description': 'None', 'username': 'client_1', 'client_version': '6.26.136', 'commands': ['changename', 'setlayer', 'updateserverdata', 'disconnect', 'connect', 'newdatasock'], 'requestables': ['PING', 'CLIENTDATA']}

Ping is: 15ms


---

# ***Servercommands***

The server can send a (client-)command to a client but the client can also send a (server-)command the server. These *servercommands* are defined by the server and they execute various scripts/functions from displaying information to changing a clients username up to shutting the server down. To prevent unauthorized usage of these commands they require different permissions. Every client gains the *'user'* permission automatically when connecting to the serer. This allows to call basic commands, mostly informational ones. *'admin'* permissions aure required for things like kicking a client or shutting the server down. Additional permission groups can be added if needed. All permissions a client has can be seen in `server.conns[*id*]['permissions]`. Every text message starting with `'s.'` and with brackets will be interpreted as servercommand. In addition to the inbuilt commands new ones can be added with `newServerCommand()`.

> ***Server.newServerCommand**(name, description, action, call_as_thread=False,needed_permission='user', params=[], optional_params=[], repeatable_param=None, category='', overwrite=False)*

Every servercommand requires a **name**, a short **description** and the **action** function that will be executed when the command gets called. With **call_as_thread** enabled, the **action** will be executed in a new thread. This is important for **actions** with a long execution time since they block the socket from receiving data until the **action** is executed. The **needed_permission** parameter states required permissions to execute this command. Multiple permissions are possible. Arguments of the servercommand are stated with *params*, *optional_params* and *repeatable_param*. Only one argument *(the last one stated)* can be repeatable. With **overwrite** existing commands can be changed. To delete a servercommand use the `delServerCommand()` function.

> ***NOTE:** Clients can gain permissions with the `'s.getrights()'` servercommand. This is more detailed in the client demonstration.*

> ***NOTE:** The **action** parameter of `newServerCommand()` always gets two arguments: the **clientid** of the calling client and the arguments for the servercommand as a list*

In [15]:
# Define additional servercommand
server.newServerCommand(
    name='s.greet',
    description="Greet someone",
    action=lambda id, args: print(f"Hello {args[0]}!"),
    call_as_thread=False,  # Disabled since action is short
    needed_permission='user',  # Default
    args=['name'])

In [16]:
client1.sendMsg("s.greet(World)")

[13:52:23]client_1: s.greet(World)
Hello World!


In [17]:
def expensive(arg1, arg2=2):
    """A simulated runtime expensive function."""

    print(f"calculating arguments '{arg1}' & '{arg2}'")
    for e in range(10000):
        _ = e**e
    server.statusInfo("Finished runtime expensive function function")

server.newServerCommand(
    name='s.expensive',
    description=('Counts to a number'),
    action=lambda id, args: expensive(*args),
    call_as_thread=False,
    args=['arg1'],
    optional_args=['arg2'])

In [18]:
client2.sendMsg("s.expensive(10)")
client2.sendMsg("s.greet(Tom)")

[13:52:26]client_2: s.expensive(10)
calculating arguments '10' & '2'


Finished runtime expensive function function
[13:52:30]client_2: s.greet(Tom)
Hello Tom!


Even through both servercommands get called at *(nearly)* the same time, the *s.greet* command gets executed a few seconds later as seen by the timestamp. Since *s.expensive* is runtime expensive it should be executed in a new thread. With the changes in the cells below both commands get executed at *(nearly)* the same time.

In [19]:
server.newServerCommand(
    name='s.expensive',
    description=("Counts to a number"),
    action=lambda id, args: expensive(*args),
    call_as_thread=True,
    args=['arg1'],
    optional_args=['arg2'],
    overwrite=True)

client2.sendMsg("s.expensive(10)")
client2.sendMsg("s.greet(Tom)")

[13:52:39]client_2: s.expensive(10)
calculating arguments '10' & '2'
[13:52:39]client_2: s.greet(Tom)
Hello Tom!


Finished runtime expensive function function


In [20]:
server.delServerCommand('s.greet')
server.delServerCommand('s.expensive')

---

# ***Requestables***

(Server-)Requestables allow Clients to request data from the server. All available requestables are defined by the server. Similar to Servercommands new requestables can be defined with `newServerRequestable()` and removed with `delServerRequestable()`. Data can be requested through every sock that uses the default *recvFunc* `recvClientData()`. *(See next Chapter for more)*. The way requests work is by sending the request together with an ID to the other party. There the request is processed and send back together with the ID. A receiver function detects the request because of the ID and puts the data in a dictionary where the original requesting function collects the data and returns it.

> ***Note:** Requestables should be written in uppercase but this is not enforced.*

In [21]:
server.newServerRequestable('THE_ANSWER_TO_EVERYTHING', [42, 'towel'], overwrite=False)
server.newServerRequestable('SERVER', lambda: str(server))

In [22]:
# Client requests data from server
client1.sendRequest(client1.clientsocks[client1.connected_addr[1]], 'THE_ANSWER_TO_EVERYTHING')

[42, 'towel']

In [23]:
client1.sendRequest(client1.clientsocks[client1.connected_addr[1]], 'SERVER')

"<Server (Demoserver), active, Serversock=('192.168.178.140', 4000)>"

In [24]:
server.delServerRequestable('SERVER')
server.delServerRequestable('THE_ANSWER_TO_EVERYTHING')

In [25]:
# Server requests data from client
server.sendRequestTo(cl2_id, 'CLIENTDATA')

{'description': 'None',
 'username': 'better name',
 'client_version': '6.26.136',
 'commands': ['changename',
  'setlayer',
  'updateserverdata',
  'disconnect',
  'connect',
  'newdatasock'],
 'requestables': ['PING', 'CLIENTDATA']}

---

# ***Datasockets***

As already mentioned the server can have multiple datasocks in addition to the serversock. These datasocks can be specified for certain purposes like transmitting of images. A datasock binds itself to a certain port and awaits new connecting sockets. The connected socket passes through a registration where it is validated that the socket belongs to a client connected to the serversock. *(Sockets that don't belong to a connected client get disconnected.)* After the registration the socket gets its own receiving thread which will receive and process data on that socket through a *recvFunc*. Every datasock can have its own *(custom)* *recvFunc* but the default is `recvClientData()`.
An important datasock is the *maindatasock*, that is used for background transmissions like requests while the serversock connects new clients and handles user actions like servercommands. The maindatasock can be initialized at server start or later on through the function `bindMaindataSock()`. Since the maindatasock wasn't initiated with serverstart we can bind it now.

In [26]:
maindatasock = server.bindMaindataSock()

Maindata Datasock bound to 3999


Other datasocks can be initialized with `newDataSock()` as shown below. The datasock can be bound to a specific **dataport** or on the first available port in a list. By default the function will use the preferred data ports set with server initialization. The **connect_clients** parameter allows to connect specific clients to the new datasock. If **connect_new_clients** is enabled future clients that connect to the serversock will automatically be connected with this datasock. The most important parameter is the **recvFunc** parameter. This will set how the datasock receives and processes data. If the recvFunc is not specified, the default `recvClientData()` function will be used.

In [27]:
datasock = server.newDataSock(
    sockname='DataSocket1',
    dataport=[3999, 3995],
    recvFunc=server.recvClientData,
    connect_clients=[cl2_id],
    connect_new_clients=True,
    show_info=True)

DataSocket1 Datasock bound to 3995


A custom build recvFunc receives and processes data. The function should be wrapped in/decorated with the `Server.recvFuncWrapper` which will loop the recvFunc. The wrapper will also handle errors and proper disconnect and removal of the socket. Keep in mind that the recvFunc gets two arguments, the socket to receive on and the clientid and also note that the client may need a custom send function.

In [28]:
# Build custom recvFunc
@ltn.Server.recvFuncWrapper
def recvFunc_bytes_size(server, socket, clientid):
    """RecvFunc that only displays the size of the received data."""

    data = socket.recv(server.se_max_datasize)
    print(f"Received {len(data)} bytes")


# Create new datasocket with custom recvFunc
custom_datasock = server.newDataSock(
    'CustomDatasock',
    lambda sock, id: recvFunc_bytes_size(server, sock, id),
    dataport=4005,
    connect_clients=[cl1_id],
    show_info=True)

CustomDatasock Datasock bound to 4005


In [29]:
_ = client1.clientsocks[4005].send("Hello".encode())

Received 5 bytes


As we see instead of the text the binary size of the data is outputted.

To close a datasock use the `closeDataSock()` function with the datasock name as argument. This disconnects every socket on that datasock and unbinds the datasock.

In [30]:
server.closeDataSock('DataSocket1')
server.closeDataSock('CustomDatasock')

Closing datasock 'DataSocket1' at 3995
Closing datasock 'CustomDatasock' at 4005


---

# ***Services***

The server has a services routine for repetitive tasks like ping checking or data synchronization. The routine can be initialized with server start or later on through `servicesController()`. Once started all initialized and enabled services will be executed iteratively. A new service can be defined with `newService()`. A service function gets a list with clientids as argument. To enable or disable a service use `setServiceEnabled()`. Services that throw an error will automatically be disabled.

In [31]:
def ping_service(clientids):
    """Pings clients"""
    for clientid in clientids:
        server.conns[clientid]['ping'] = server.pingSocket(
            server.mainDataSockof(clientid))

server.newService('ping', ping_service, as_thread=False, overwrite=False)

In [32]:
server.servicesController(routine_pause=1)

Started services


In [33]:
# Rerun this multiple times to see the ping change
server.conns[cl1_id]['ping']

0.03125

In [34]:
server.delService('ping') 

---

# ***Server shutdown***

The server can be shut down through `shutdownServer()`. This will disconnect all clients *(on every sock)*, end the services routine and unbinds all socks of the server. It is also possible to restart the server with `restartServer()`. Both functions can be called through servercommands.

In [35]:
server.shutdownServer()

Shutting Server down
Stopping services
Disconnecting all users
Disconnected connection 'client_1'
Disconnected connection 'client_2'
Unbinding all sockets
Closing datasock 'Serversock' at 4000
Closing datasock 'Maindata' at 3999
Server is inactive


Ended services


---

# ***Outputs***

The server outputs different messages through the following functions. The *Info functions simply print the information. The output functions can be modified if needed.

> ***Server.printInfo(message: str, sender: str)***
> * Outputs received server messages

> ***Server.connectionInfo(info: str)***
> * Outputs information regarding clients

> ***Server.statusInfo(status: str)***
> * Outputs states of the server

> ***Server.warningInfo(warning: str)***
> * Outputs warnings

> ***Server.errorInfo(error: str)***
> * Outputs errors

> ***Server.logEvent(event: str)***
> * Stores every occurred output internally and in the *logfile* if one is set


---
<span style="float: right; font-size: 15pt"><b>LocalThingsNetwork</b></i></span>
<br><br>
<span style="float: right; font-size: 10pt"><i>For Server version </i><b>7.43.162</b></span>